In [1]:
import numpy as np
import pandas as pd
import os 
import random 
import matplotlib.pyplot as plt 
import copy
import scipy
from PIL import Image

In [2]:
"""
OVERVIEW: 
We are going to implement a N-layer deep neural network ... so ... the input at first will be how many layers and the 
number of nodes in each layer... 

using that we will randomly initialize our parameter...  compete forward propagation using ReLU activation for every layer
except last where we will use sigmoid activation 
than we will calculate cost ...  and update our parameters calculating the derivatives using bacward propagation

combining all these above we then will build our model to train and predict 
"""

#data reading and pre processing and assigning initial values 

#setting the path for the folder to read ... 
path1 = 'F:\code files\machine learning\dogs vs cats classification\dataset'

#setting important values of image
height = 64
width = 64


In [3]:
def load_location(directory):
    biglist = []
    for folder in os.listdir(directory):
        for file in os.listdir(os.path.join(directory,folder)):
            location = os.path.join(directory,folder,file)
            biglist.append([location,folder])
    return biglist

In [4]:
def load_dataset(path):
    #print(path)
    #print("we are not in except also")
    image = np.array(Image.open(path))
    image = np.resize(image, (height,width,3))
    image = image.astype('float32')
    image /= 255
    return image

In [5]:
#loading dataset of training images 
X_train_list = []
Y_train_list = []
directory = os.path.join(path1, 'Train')
biglist = load_location(directory)
for i in range(len(biglist)):
    item = random.choice(biglist)
    image_array = load_dataset(item[0])
    X_train_list.append(image_array)
    if item[1] == 'Dog':
        Y_train_list.append(1)
    else:
        Y_train_list.append(0)
    biglist.remove(item)
print("training images loading done")

#loading dataset of testing images
X_test_list = []
Y_test_list = []
directory = os.path.join(path1, 'Test')
biglist = load_location(directory)
for i in range(len(biglist)):
    item = random.choice(biglist)
    image_array = load_dataset(item[0])
    X_test_list.append(image_array)
    if item[1] == 'Dog':
        Y_test_list.append(1)
    else:
        Y_test_list.append(0)
    biglist.remove(item)
print("testing images loading done")

#finding number of training and testing examples
m_train = len(Y_train_list)
m_test = len(Y_test_list)

#converting our list data to numpy array and then flattening X value to (height*width*3 , 1) shape
X_train = np.asarray(X_train_list).reshape(m_train, -1).T
X_test = np.asarray(X_test_list).reshape(m_test, -1).T
Y_train = np.asarray(Y_train_list).reshape(m_train,1).T
Y_test = np.asarray(Y_test_list).reshape(m_test,1).T

C:\Users\hp\anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:793: UserWarning: Truncated File Read
  warnings.warn(str(msg))


training images loading done
testing images loading done


In [6]:
#the model begins 
#first some helper functions 

#initializes random weights and zeros to w and b of all layer of our network
def initialize_parameters(dims):
    parameters = {}
    L = len(dims)
    for l in range(1,L):
        parameters['W'+str(l)] = np.random.randn(dims[l],dims[l-1])*0.01
        parameters['b'+str(l)] = np.zeros((dims[l],1))
    return parameters

In [7]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

In [8]:
def relu(z):
    s = np.maximum(0,z)
    return s

In [9]:
def sigmoid_backward(z):
    s = sigmoid(z) * (1-sigmoid(z))
    return s

In [10]:
def relu_backward(z):
    s = np.where(z>0, 1, 0)
    return s

In [11]:
def forward_propagation(X,parameters):
    A = X
    L = len(parameters)//2
    
    #a accumulator to hold information
    caches = []
    
    for l in range(1,L):
        cache = {}
        A_old = A
        W = parameters['W'+str(l)] 
        b = parameters['b'+str(l)]
        #print(f"W{l} {W.shape}  X  {A_old.shape} relu ")
        Z = np.dot(W, A_old)+b
        A = relu(Z)
        cache['Z'] = Z
        cache['A_prev'] = A_old
        cache['W'] = W 
        cache['A'] = A
        caches.append(cache)
    
    cache = {}
    A_old = A
    W = parameters['W'+str(L)]
    b = parameters['b'+str(L)]
    #print(f"W{L} {W.shape}  X  {A_old.shape} sigmoid")
    Z = np.dot(W, A_old)+b
    A = sigmoid(Z)
    #print(A.shape)
    cache['Z']  = Z
    cache['A_prev'] = A_old
    cache['W']  = W
    cache['b'] = b
    cache['A'] = A
    caches.append(cache)
         
    return A,caches

In [12]:
def cost(AL , Y):
    m = Y.shape[1]
    cost = np.add(np.dot(Y, np.log(AL).T), np.dot((1-Y), np.log(1-AL).T))/(-m)
    J = np.squeeze(cost)
    #print(J.shape)
    return J
    

In [13]:
def linear_back(cache, dA,  activation):
    Z = cache['Z']
    A_prev = cache['A_prev']
    W = cache['W']
    
    #print(Z.shape ,'Z shape')
    #print(dA.shape , 'da shape')
    #print(A_prev.shape , ' A prev shape')
    
    
    m = A_prev.shape[1]
    if activation == 'relu':
        dZ = dA * relu_backward(Z)
    elif activation == 'sigmoid':
        dZ = dA * sigmoid_backward(Z)
        #print(A_prev.shape , 'A_prev shape')
    dW = np.dot(dZ, A_prev.T)/m
    #print(dW.shape, 'dw shape')
    #print(W.shape, 'W shape')
    #print(activation)
    db = np.sum(dZ, axis = 1, keepdims = True)/m
    dA_prev = np.dot(W.T, dZ)
    
    return dA_prev, dW, db
    
    

In [14]:
def back_propagation(caches, Y, Al ):
    #print(len(caches))
    L = len(caches)
    m = Al.shape[1]
    dAl = - (np.divide(Y,Al) - np.divide((1-Y),(1-Al)))
    grads = {}
    
    cache = caches[L-1]
    dA_prev , dW, db = linear_back(cache, dAl,  'sigmoid')
    
    grads['dA'+str(L-1)] = dA_prev
    grads['dW'+str(L)] = dW
    grads['db'+str(L)] = db
    
    for l in reversed(range(L-1)):
        cache = caches[l]
        dA_prev , dW, db = linear_back(cache,dA_prev, 'relu')
        grads['dA'+str(l)] = dA_prev
        grads['dW'+str(l+1)] = dW
        grads['db'+str(l+1)] = db
    
    return grads

In [15]:
def update_parameters(parameters, grads, learning_rate):
    parameters = parameters.copy()
    L = len(parameters)//2
    for l in range(L): 
        parameters['W'+str(l+1)] = parameters['W'+str(l+1)] - learning_rate * grads['dW'+str(l+1)]
        parameters['b'+str(l+1)] = parameters['b'+str(l+1)] - learning_rate * grads['db'+str(l+1)]
        
    return parameters
    

In [16]:
def predict(X, parameters):
    Al, caches = forward_propagation(X, parameters)
    predictions = np.where(Al>0.5, 1, 0)
    return predictions
    

In [17]:
def model(X_train, Y_train, X_test, Y_test , layer_dimension, learning_rate = 0.001, num_of_iterations = 1000):
    parameters = initialize_parameters(layer_dimension)
    costs = []
    
    for i in range(num_of_iterations):
        Al, caches = forward_propagation(X_train, parameters)
        J = cost(Al, Y_train)
        if i % 100 == 0:
            print(f"the cost after {i}th iteration is {J}")
        costs.append(J)
        grads = back_propagation(caches , Y_train , Al)
        parameters = update_parameters(parameters, grads, learning_rate)
        
    train_predictions = predict(X_train, parameters)
    test_predictions = predict(X_test, parameters)
    
    train_accuracy = 100 - np.mean(np.abs(train_predictions - Y_train))*100
    test_accuray = 100 - np.mean(np.abs(test_predictions - Y_test))*100
    
    print(f"train accuracy is {train_accuracy}")
    print(f"test accuray is {test_accuray}")
    
    return parameters, costs
    

In [18]:
features = height * width * 3
layer_dimension = [features, 5,4,1]
parameters, costs = model(X_train, Y_train, X_test, Y_test , layer_dimension, learning_rate = 0.001, num_of_iterations = 1000)
#print(parameters, costs)

the cost after 0th iteration is 0.6931469801506738
the cost after 100th iteration is 0.6931469790648933
the cost after 200th iteration is 0.693146977942565
the cost after 300th iteration is 0.6931469767629259
the cost after 400th iteration is 0.6931469755733445
the cost after 500th iteration is 0.6931469743861299
the cost after 600th iteration is 0.6931469731922535
the cost after 700th iteration is 0.6931469719817984
the cost after 800th iteration is 0.693146970750666
the cost after 900th iteration is 0.6931469695065473
train accuracy is 50.002427302296226
test accuray is 50.0
